In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import numpy as np
from point import Point

def get_cpk_boundary():
    with open('cpk_boundary.pickle', 'rb') as handle:
        return pickle.load(handle)

def get_trip_class(boundary, row):
    # point inside: 0, 1, 2
        # 0: outside polygon
        # 1: inside polygon
        # 2: on the polygon

    start = Point(row['START LAT'], row['START LONG'])
    end = Point(row['END LAT'], row['END LONG'])
    start_campus = boundary.point_inside(start)
    end_campus = boundary.point_inside(end)

    # Trip classes:
        # A = start and end on campus
        # B = start on campus, end off campus
        # C = start off campus, end on campus

    if start_campus >= 1 and end_campus >= 1:
        return 'A'
    elif start_campus >= 1 and end_campus == 0:
        return 'B'
    elif start_campus == 0 and end_campus >= 1:
        return 'C'
    else:
        return 'D'

In [2]:
df = pd.read_csv('scooters.csv')
df = df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'PATH', 'TIMESTAMPS'], axis=1)

boundary = get_cpk_boundary()
df['TRIP_CLASS'] = df.apply (lambda row: get_trip_class(boundary, row), axis=1)
# df.to_csv('RidesWithTripClass.csv')

In [3]:
df1 = df[df['TRIP_CLASS'] == 'A']
df2 = df[df['TRIP_CLASS'] == 'B']
df3 = df[df['TRIP_CLASS'] == 'C']
df4 = df[df['TRIP_CLASS'] == 'D']

df12019 = df1[df1['CREATED'].str.startswith('2019')]
df12020 = df1[df1['CREATED'].str.startswith('2020')]
df22019 = df2[df2['CREATED'].str.startswith('2019')]
df22020 = df2[df2['CREATED'].str.startswith('2020')]
df32019 = df3[df3['CREATED'].str.startswith('2019')]
df32020 = df3[df3['CREATED'].str.startswith('2020')]
df42019 = df4[df4['CREATED'].str.startswith('2019')]
df42020 = df4[df4['CREATED'].str.startswith('2020')]


df2019 = df[df['CREATED'].str.startswith('2019')]
df2020 = df[df['CREATED'].str.startswith('2020')]

## Creates csv files from dataframes

# df12019.to_csv('RidesWithTripClassA2019.csv')
# df12020.to_csv('RidesWithTripClassA2020.csv')
# df22019.to_csv('RidesWithTripClassB2019.csv')
# df22020.to_csv('RidesWithTripClassB2020.csv')
# df32019.to_csv('RidesWithTripClassC2019.csv')
# df32020.to_csv('RidesWithTripClassC2020.csv')
df42019.to_csv('RidesWithTripClassD2019.csv')
df42020.to_csv('RidesWithTripClassD2020.csv')


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def get_day(row):
    return int(row.split(' ')[0].split('-')[2])

def get_plot(df):
    df['DAY'] = df['CREATED'].apply(lambda row: get_day(row))
    df.sort_values('DAY', inplace=True)
    return df['DAY'].value_counts().sort_index(axis=0)

def get_ride_plot(year):
    dfA = pd.read_csv(f'RidesWithTripClassA{year}.csv')
    dfB = pd.read_csv(f'RidesWithTripClassB{year}.csv')
    dfC = pd.read_csv(f'RidesWithTripClassC{year}.csv')
    dfD = pd.read_csv(f'RidesWithTripClassD{year}.csv')

    tripA = 'Trip starts and ends on campus'
    tripB = 'Trip starts on campus and ends off campus'
    tripC = 'Trip starts off campus and ends on campus'
    tripD = 'Trip starts and ends off campus'
    dict = {tripA: get_plot(dfA), tripB: get_plot(dfB), tripC: get_plot(dfC), tripD: get_plot(dfD)}
    data = pd.DataFrame(dict)
    ax = data.plot(kind='bar', stacked=True, rot=0, figsize=(8,5))
    dir = 'upper right' if year == 2020 else 'upper left'
    ax.legend(loc=dir)
    plt.xlabel(f'Day of October {year}')
    plt.ylabel('Number of Rides')
    plt.title('Number of Rides per Trip Class')
    sns.despine()
    plt.tight_layout()
    plt.show()

In [5]:
get_ride_plot(2019)

FileNotFoundError: [Errno 2] No such file or directory: 'RidesWithTripClassA2019.csv'

In [ ]:
get_ride_plot(2020)